In [13]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re

In [14]:
import numpy as np
import pandas as pd

df = pd.read_csv('airline-tweets-sentiments.csv')
df.columns = ['id', 'tweets', 'language', 'sentiment_value']

In [15]:
all_words = []
doc = []
from nltk.corpus import stopwords
import re

stop_words = list(set(stopwords.words('english')))

allowed_word_types = ["J"]

for index, row in df.iterrows():
    if row['sentiment_value'] == 2: 
        doc.append( (row['tweets'], "pos") )

        cleaned = re.sub(r'[^(a-zA-Z)\s]','', row['tweets'])

        tokenized = word_tokenize(cleaned)

        stopped = [w for w in tokenized if not w in stop_words]

        pos = nltk.pos_tag(stopped)

        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())

    elif row['sentiment_value'] == 0:
            doc.append( (row['tweets'], "neg") )
            cleaned = re.sub(r'[^(a-zA-Z)\s]','', row['tweets'])
            tokenized = word_tokenize(cleaned)
            stopped = [w for w in tokenized if not w in stop_words]
            neg = nltk.pos_tag(stopped)
            for w in neg:
                if w[1][0] in allowed_word_types:
                    all_words.append(w[0].lower())


In [34]:
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:5000]

def find_features(doc):
    words = word_tokenize(doc)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

featuresets = [(find_features(rev), category) for (rev, category) in doc]

random.shuffle(featuresets)

training_set = featuresets[:300]
testing_set = featuresets[300:]


In [35]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(15)

Classifier accuracy percent: 81.35593220338984
Most Informative Features
                 luggage = True              neg : pos    =      8.4 : 1.0
                   thank = True              pos : neg    =      6.3 : 1.0
                    free = True              pos : neg    =      6.3 : 1.0
                   delta = True              neg : pos    =      5.5 : 1.0
                 amazing = True              pos : neg    =      4.9 : 1.0
                     bag = True              neg : pos    =      4.9 : 1.0
                  airbus = True              pos : neg    =      3.5 : 1.0
                    want = True              pos : neg    =      3.5 : 1.0
                    good = True              pos : neg    =      3.0 : 1.0
                    cool = True              pos : neg    =      3.0 : 1.0
                   happy = True              pos : neg    =      3.0 : 1.0
                   cabin = True              pos : neg    =      3.0 : 1.0
                     klm = 